In [ ]:
#Import scikit-learn dataset library
from sklearn import datasets
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix, mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
from math import sqrt
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")
 
from datetime import datetime
start_time = datetime.now()

In [ ]:
# df= pd.read_excel('F://UCSI_research\PG_Matters\Chan Kok Pin\post-viva\Dataset\Regression_data\Grain_online\grain_moisture.xlsx')
df = pd.read_excel('F://UCSI_research\PG_Matters\Chan Kok Pin\post-viva\Dataset\Regression_data\Grain_online\grain_moisture_snv.xlsx')
 
# from pyspectra.transformers.spectral_correction import msc, detrend ,sav_gol,snv
# MSC= msc()
# MSC.fit(df_raw)
# df=MSC.transform(df_raw)
    
    
    
X = df.iloc[: , 1:]
Y = df.iloc[:, 0]


# print(len(Y))
# print(len(X))
print(X.shape)
print(Y.shape)

In [ ]:
#  #add print spectrum...
# import seaborn as sns
# sns.set_theme()

# X1 = df.values[:, 1:].astype('float32')  
# y_toc = df.values[:,0]    
# wl = np.linspace(1104,2496, num=X1.shape[1], endpoint=True)   #wavelength range
# plt.plot(wl,X1.T)
# plt.title('Moisture of Grain Protein (MGP)', fontsize = 12)
# plt.xlabel(r'Wavelength(nm)', fontsize=12)
# plt.ylabel('Absorbance(au)', fontsize=12)
# plt.savefig('MGP tablet.png', dpi=300)
# plt.show()

In [ ]:
df.head()

In [ ]:
sns.boxplot(x=df["Y"])
plt.title('Moisture of Grain Protein (MGP)', fontsize = 12)
plt.xlabel('Moisture content (%)')

In [ ]:
#train set,test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)  

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
# Fitting Decision Tree Regression to the dataset
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

DT = DecisionTreeRegressor()
RF = RandomForestRegressor()
LGB=lgb.LGBMRegressor(verbose=0)

In [ ]:
# #tuning   hyperparameters
paramsDT={"splitter":["best","random"],
            "max_depth" : [1,3,5,7,9,11,12],
           "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
           "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
           "max_features":["auto","log2","sqrt",None],
           "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] }

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 800, num = 8)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
paramsRF = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


paramsLGB={'n_estimators':[100,200,300,400,500], 'learning_rate':[0.1,0.25,0.5,0.75, 1],'reg_alpha':[0,0.01,0.05,0.1]}


# TUNING DT

In [ ]:
DT1 = RandomizedSearchCV(estimator = DT, param_distributions = paramsDT, n_iter = 10, 
                               cv = 3, verbose=3, random_state=0, n_jobs = -1)

DT1.fit(X_train, Y_train)

In [ ]:
best_DT = DT1.best_estimator_
print(best_DT)


In [ ]:
# train the model with its best paramaters
best_DT.fit(X_train, Y_train)

In [ ]:
dt_p1 = best_DT.predict(X_train)
dt_p2 = best_DT.predict(X_test)
print('DT result')
print('train R^2: %.2f' % r2_score(Y_train, dt_p1))
print('train MAE: %.2f' % mean_absolute_error(Y_train, dt_p1))
print('test  R^2: %.2f' % r2_score(Y_test, dt_p2))
print('test  MAE: %.2f' % mean_absolute_error(Y_test, dt_p2))

In [ ]:
# plot prediction
sns.set_theme()
plt.scatter(Y_test,dt_p2)
x=np.linspace(5,25,10)
y=x
plt.plot(x,y,color='red',linewidth=1.0,linestyle='--',label='line')
#plt.legend(["y = x","predict"])
 
plt.title("Moisture of Grain Protein (MGP)")
plt.xlabel('Observed MGP (%)')
plt.ylabel('Predicted MGP (%)')
plt.savefig('MGP-graph.png', dpi=300, bbox_inches='tight', transparent=False)
plt.show()


# TUNING LGBM

In [ ]:
LGB1 = RandomizedSearchCV(estimator = LGB, param_distributions = paramsLGB, n_iter = 10, 
                               cv = 3, verbose=3, random_state=0, n_jobs = -1)

LGB1.fit(X_train, Y_train)

In [ ]:
best_LGB = LGB1.best_estimator_
print(best_LGB)


In [ ]:
# train the model with its best paramaters
best_LGB.fit(X_train, Y_train)

In [ ]:
lgb_p1 = best_LGB.predict(X_train)
lgb_p2 = best_LGB.predict(X_test)
print('LGBM result')
print('train R^2: %.2f' % r2_score(Y_train, lgb_p1))
print('train MAE: %.2f' % mean_absolute_error(Y_train, lgb_p1))
print('test  R^2: %.2f' % r2_score(Y_test, lgb_p2))
print('test  MAE: %.2f' % mean_absolute_error(Y_test, lgb_p2))
print('test  RMSE: %.2f' %  sqrt(mean_squared_error(Y_test, lgb_p2)))

In [ ]:
plt.scatter(Y_test,lgb_p2)
x=np.linspace(5,25,10)
y=x
plt.plot(x,y,color='red',linewidth=1.0,linestyle='--',label='line')
#plt.legend(["y = x","predict"])
 
plt.title("Moisture of Grain Protein (MGP)")
plt.xlabel('Observed MGP (%)')
plt.ylabel('Predicted MGP (%)')
plt.savefig('MGP-graph.png', dpi=300, bbox_inches='tight', transparent=False)
plt.show()


# TUNING RF

In [ ]:
RF1 = RandomizedSearchCV(estimator = RF, param_distributions = paramsRF, n_iter = 20, 
                               cv = 3, verbose=3, random_state=0, n_jobs = -1)

RF1.fit(X_train, Y_train)


In [ ]:
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
best_RF = RF1.best_estimator_
print(best_RF)


In [ ]:
# train the model with its best paramaters
best_RF.fit(X_train, Y_train)

In [ ]:
rf_p1 = best_RF.predict(X_train)
rf_p2 = best_RF.predict(X_test)
print('RF result')
print('train R^2: %.2f' % r2_score(Y_train, rf_p1))
print('train MAE: %.2f' % mean_absolute_error(Y_train, rf_p1))
print('test  R^2: %.2f' % r2_score(Y_test, rf_p2))
print('test  MAE: %.2f' % mean_absolute_error(Y_test, rf_p2))
print('test  RMSE: %.2f' %  sqrt(mean_squared_error(Y_test, rf_p2)))

In [ ]:
plt.scatter(Y_test,rf_p2)
x=np.linspace(5,25,10)
y=x
plt.plot(x,y,color='red',linewidth=1.0,linestyle='--',label='line')
#plt.legend(["y = x","predict"])
 
plt.title("Moisture of Grain Protein (MGP)")
plt.xlabel('Observed MGP (%)')
plt.ylabel('Predicted MGP (%)')
plt.savefig('MGP-graph.png', dpi=300, bbox_inches='tight', transparent=False)
plt.show()
